In [1]:
import pandas as pd
import numpy as np

# https://dash.plotly.com/installation
# https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e
from sqlalchemy import create_engine
import mysql.connector #  pip3 install mysql-connector-python
import configparser as cp

## Functions 

In [20]:
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

# DB connection

In [2]:
config=cp.ConfigParser()
config.read('/home/ubuntu/cert/db_login.txt')
db_config=config['ivan_db']


## 2. db connection
engine=create_engine('mysql+mysqlconnector://{0:s}:{1:s}@{2:s}/{3:s}'.format(db_config['userid'],
                                                                             db_config['pwd'],
                                                                             db_config['hostname'],
                                                                             'STOCK_PRED'
                                                                            ))

### Overall Stock 

In [3]:
## Data loading
df=pd.read_sql("""SELECT *
                  FROM STOCK_PRED.ALL_STOCK_HIST
                  WHERE DATE>=CURDATE()-INTERVAL 360 DAY""",
              con=engine)
print(df.shape)
print(df.Stock.nunique())
print(df.REFRESH_DATE.max())
df.tail(5)

(1423581, 9)
5991
2022-04-12 02:53:37


,Date,Open,High,Low,Close,Volume,SE,Stock,REFRESH_DATE
1423576,2022-04-11,9.620000,9.790000,9.600000,9.70,109700.0,NYSE,ZTR,2022-04-12 02:53:37
1423577,2022-04-11,198.970001,199.880005,192.009995,192.50,2252200.0,NYSE,ZTS,2022-04-12 02:53:37
1423578,2022-04-11,13.590000,13.880000,13.400000,13.59,1184700.0,NYSE,ZUO,2022-04-12 02:53:37
1423579,2022-04-11,4.250000,4.250000,4.070000,4.17,135300.0,NYSE,ZVIA,2022-04-12 02:53:37
1423580,2022-04-11,6.390000,6.500000,5.840000,6.08,1031100.0,NYSE,ZYME,2022-04-12 02:53:37


### Linear regression summary table 

In [5]:
df_lr=pd.read_sql("""SELECT *
                     FROM STOCK_PRED.LINEAR_REG_L40
                     WHERE MODEL_DATE>STR_TO_DATE('2022-04-11','%Y-%m-%d')
                     ORDER BY MODEL_DATE DESC, WT_COEF DESC""",
                  con=engine
                 )
print(df_lr.shape)
print(df_lr.Stock.nunique())
df_lr.groupby('Model_date').size()

(5581, 14)
5581


Model_date
2022-04-11 19:55:00    5581
dtype: int64

In [6]:
df_lr.head(5)

,Model_date,Stock,Name,Industry,R_squared,Coef,P_values,WT_Coef,Start_Date,End_Date,start_price,end_price,Num_records_dist,growth_rate
0,2022-04-11 19:55:00,TPL,Texas Pacific Land Corporation Common Stock,Oil & Gas Production,0.836980,11.809988,1.507742e-16,9.884722,2022-02-11,2022-04-11,1105.449951,1390.829956,40,0.258157
1,2022-04-11 19:55:00,MKL,Markel Corporation Common Stock,Property-Casualty Insurers,0.831241,8.570447,1.183984e-16,7.124106,2022-02-11,2022-04-11,1242.000000,1477.199951,41,0.189372
2,2022-04-11 19:55:00,AZO,AutoZone Inc. Common Stock,None,0.623118,6.412654,8.632706e-10,3.995840,2022-02-11,2022-04-11,1949.109985,2158.479980,41,0.107418
3,2022-04-11 19:55:00,Y,Alleghany Corporation Common Stock,Property-Casualty Insurers,0.626800,6.231434,7.109114e-10,3.905863,2022-02-11,2022-04-11,670.570007,844.690002,41,0.259660
4,2022-04-11 19:55:00,TSLA,Tesla Inc. Common Stock,Auto Manufacturing,0.556753,6.883548,2.149365e-08,3.832436,2022-02-11,2022-04-11,860.000000,975.929993,41,0.134802


In [46]:
px.scatter(df_lr,
           x='Coef',y='R_squared',color='growth_rate',
           title='Sotck Linear Regression Analysis'
                     )

# Dashboard
- https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e

## import packages 

In [28]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
#import dash_html_components as html
from dash import html, dash_table
from dash.dependencies import Input,Output

## Dashboard body - 1

In [17]:
# buid app
app=JupyterDash(__name__)

fig=px.scatter(df_lr,
           x='R_squared',y='Coef',color='growth_rate',
           title='Sotck Linear Regression Analysis'
                     )

app.layout=html.Div([html.H1('Stock Linear Regression Analysis'),
                     dcc.Graph(id='graph',
                               figure=fig
                              )])





## run app
app.run_server(mode='external',host='ec2-52-14-195-139.us-east-2.compute.amazonaws.com',debug=True)

KeyboardInterrupt: 

## Dashboard body -2 

In [33]:
app=JupyterDash(__name__)

ln_reg_plot=px.scatter(df_lr,
                       x='R_squared',y='Coef'#,size='growth_rate',color='Industry'#
                       ,hover_name='Name'
                      )

app.layout=html.Div([
    html.Div([
        html.H1('Summary Table'),
        dash_table.DataTable(df_lr.head(10).to_dict('records'),
                             [{'name':i, 'id':i} for i in df_lr.columns]
                            )
    ],style={'display':'inline-block'}),
    html.Div([
        html.H1('Coef VS R_squared Distribution'),
        dcc.Graph(figure=ln_reg_plot,style={'display':'inline-block'})
        
    ])
])

if __name__=='__main__':
    app.run_server(mode='external',
                   host='ec2-3-22-120-133.us-east-2.compute.amazonaws.com',
                   debug=True)

/home/ubuntu/anaconda3/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://ec2-3-22-120-133.us-east-2.compute.amazonaws.com:8050/


In [26]:
#app._terminate_server_for_port("ec2-3-22-120-133.us-east-2.compute.amazonaws.com", 8050)